# Low light Image enhancement 
In this notebook, we illustrate the use of the Zero-dce model for low light image enhancement using custom postprocessor with DeGirum PySDK

### Define custom postprocessor

In [ ]:
from degirum.postprocessor import InferenceResults
from degirum._model_param_helpers import model_shape_get
import numpy as np
import cv2

class EnhancementPostprocessor(InferenceResults):
    """Post‑processor that converts a flattened RGB tensor `(1, H×W, 3)`
    produced by low‑light enhancement models into a displayable `uint8` BGR image.

    After construction:
    * ``self._enhanced_image`` holds the processed frame (OpenCV format).
    * ``self._inference_results`` is replaced with a one‑liner metadata dict.
    * ``image`` and ``image_overlay`` both return the enhanced image so the usual
      SDK viewers work out of the box.
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        try:
            raw_tensor = np.asarray(self._inference_results[0]["data"], dtype=np.float32)
        except Exception as e:  # pragma: no cover – developer mistake guard
            raise RuntimeError(
                "EnhancementPostprocessor expects inference_results[0]['data'] to "
                "contain the flat output tensor.  Didn't find it: " + str(e)
            ) from e

        raw_tensor = raw_tensor.squeeze(0)

        try:
            # InputShape is typically (1, C, H, W) or (1, H, W, C)
            _, H, W, _ = model_shape_get(self._model_params, 0, 4)
        except Exception:
            H, W = 400, 600  # Zero‑DCE reference resolution

        if raw_tensor.shape[0] != H * W:
            raise ValueError(
                f"Flat tensor length {raw_tensor.shape[0]} does not match H×W "
                f"({H}×{W}={H*W}).  Check model output/shape assumptions."
            )

        img = raw_tensor.reshape(H, W, 3)       
        self._enhanced_image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        self._inference_results = [
            {
                "category_id": 0,
                "label": "enhanced_frame",
                "score": 1.0,
            }
        ]

    @property
    def image(self):
        """Return the post‑processed frame."""
        return self._enhanced_image

    @property
    def image_overlay(self):  # no overlay drawing needed
        return self._enhanced_image



In [ ]:
import degirum as dg
import cv2

inference_host_address = "@local"
zoo_url = '../models'
token=''
device_type='HAILORT/HAILO8'
model_name = 'zero_dce--400x600_quant_hailo_hailo8_1'

model = dg.load_model(
    model_name=model_name,
    inference_host_address=inference_host_address,
    zoo_url=zoo_url,
    token=token,
    device_type=device_type,
    custom_postprocessor= EnhancementPostprocessor, # Custom postprocessor
)


# Load an image
enh_inf = model('../assets/dark_room.jpg')

# Save image overlay output
enhanced_frame = enh_inf.image_overlay if hasattr(enh_inf, "image_overlay") else enh_inf.image
cv2.imwrite("enhanced_image.jpg", enhanced_frame)
print("Enhanced image saved as 'enhanced_image.jpg'")